In [4]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
#from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.model_selection import GridSearchCV  #Perforing grid search

In [5]:
# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/xgboost-dm'
my_region = boto3.session.Session().region_name # set the region of the instance

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")

print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + xgboost_container + " container for your SageMaker endpoint.")

Success - the MySageMakerInstance is in the us-east-1 region. You will use the 811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [6]:
bucket_name = 'credit-card-kaggle'
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    else: 
        s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


In [7]:
bucket='cc-data-kaggle'
data_key = 'creditcard.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)

df = pd.read_csv(data_location)

In [8]:
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [10]:
train_data, test_data = np.split(df.sample(frac=1, random_state=1729), [int(0.7 * len(df))])
print(train_data.shape, test_data.shape)

(199364, 31) (85443, 31)


In [22]:
bucket_name = 'credit-card-kaggle'
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


In [23]:
try:
  urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
  print('Success: downloaded bank_clean.csv.')
except Exception as e:
  print('Data load error: ',e)

try:
  model_data = pd.read_csv('./bank_clean.csv',index_col=0)
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [34]:
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


### Modeling

In [25]:
#pd.concat([train_data['Class'], train_data.drop(['Class'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [26]:
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(xgboost_container,role, instance_count=1, instance_type='ml.m4.xlarge',output_path='s3://{}/{}/output'.format(bucket_name, prefix),sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,eta=0.2,gamma=4,min_child_weight=6,subsample=0.8,silent=0,objective='binary:logistic',num_round=100)

In [27]:
xgb.fit({'train': s3_input_train})

2022-04-16 19:50:23 Starting - Starting the training job...
2022-04-16 19:50:38 Starting - Preparing the instances for trainingProfilerReport-1650138623: InProgress
.........
2022-04-16 19:52:15 Downloading - Downloading input data......
2022-04-16 19:53:15 Training - Downloading the training image...
2022-04-16 19:53:51 Training - Training image download completed. Training in progress..Arguments: train
[2022-04-16:19:53:54:INFO] Running standalone xgboost training.
[2022-04-16:19:53:54:INFO] Path /opt/ml/input/data/validation does not exist!
[2022-04-16:19:53:54:INFO] File size need to be processed in the node: 3.38mb. Available memory size in the node: 8496.59mb
[2022-04-16:19:53:54:INFO] Determined delimiter of CSV input is ','
[19:53:54] S3DistributionType set as FullyReplicated
[19:53:54] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[19:53:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes

### Deployment

In [28]:
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

----------!

In [29]:
test_data

,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
10498,29,2,999,0,1,0,0,1,0,0,...,0,0,0,1,0,0,1,0,1,0
9988,30,10,999,0,1,0,0,0,0,0,...,0,0,0,0,1,0,1,0,1,0
10149,32,1,999,0,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
38365,25,1,999,0,1,0,1,0,0,0,...,0,1,0,0,0,0,1,0,1,0
18230,35,2,999,0,1,0,0,0,0,0,...,0,0,0,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23385,29,3,999,0,1,0,1,0,0,0,...,0,0,0,0,1,0,1,0,1,0
38797,60,2,6,1,0,1,0,0,0,0,...,0,0,1,0,0,0,0,1,0,1
1267,36,4,999,0,1,0,1,0,0,0,...,0,0,1,0,0,0,1,0,1,0
3174,42,1,999,0,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1


In [31]:
from sagemaker.serializers import CSVSerializer

test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array)#.decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


In [32]:
predictions_array

array([0.04957985, 0.05882718, 0.04969021, ..., 0.03834436, 0.02422529,
       0.03714524])

### Model evaluation

In [33]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.5%

Predicted      No Purchase    Purchase
Observed
No Purchase    90% (10769)    37% (167)
Purchase        10% (1133)     63% (288) 



### Turnoff Sagemaker

In [36]:
xgb_predictor.delete_endpoint(delete_endpoint_config=True)

In [37]:
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': 'SRMZ2F7V2NQ055MJ',
   'HostId': 'WIiPgBFMyCGZIUaGlI81avhM79jaBIpWBFxWto85O4MA573ZArMcqRGRsNCddLo0pR0zZpKyDpc=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'WIiPgBFMyCGZIUaGlI81avhM79jaBIpWBFxWto85O4MA573ZArMcqRGRsNCddLo0pR0zZpKyDpc=',
    'x-amz-request-id': 'SRMZ2F7V2NQ055MJ',
    'date': 'Sat, 16 Apr 2022 20:14:08 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'sagemaker/xgboost-dm/output/xgboost-2022-04-16-19-20-24-768/rule-output/ProfilerReport-1650136824/profiler-output/profiler-report.html'},
   {'Key': 'sagemaker/xgboost-dm/output/xgboost-2022-04-16-19-50-23-042/rule-output/ProfilerReport-1650138623/profiler-output/profiler-reports/StepOutlier.json'},
   {'Key': 'sagemaker/xgboost-dm/output/xgboost-2022-04-16-19-20-24-768/rule-output/ProfilerReport-1650136824/profiler-output/profiler-reports/Low